# RO reference
Fit RO on observations to get "ground truth" behavior

## Imports

In [ ]:
import warnings
import copy
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import numpy as np
import seaborn as sns
import xarray as xr
import tqdm
import pathlib
import cmocean
import pandas as pd
import os
import scipy.stats

# Import custom modules
import src.XRO
import src.XRO_utils
import src.utils

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## get filepaths
DATA_FP = pathlib.Path(os.environ["DATA_FP"])
SAVE_FP = pathlib.Path(os.environ["SAVE_FP"])

## get RNG
rng = np.random.default_rng()

## Load data

In [ ]:
## specify filepaths
oras_fp = DATA_FP / "oras5"

## load data
data = src.utils.load_oras_spatial_extended(
    oras_fp, varnames=["tos", "ssh", "d20", "mld001"]
)

## compute indices
idx_total = src.utils.get_RO_indices(data, h_var="d20")

## Detrend / pre-process

In [ ]:
## estimate forced signal by removing 2nd-order polynomial from each calendar month
detrend_fn = lambda x: src.utils.detrend_dim(x, dim="time", deg=3)
idx = idx_total.groupby("time.month").map(detrend_fn)
idx_forced = idx_total - idx

## standardize for convenience
idx /= idx.std()

## Cross correlation stats

Plotting function

In [ ]:
def format_xcorr_ax(ax):
    """make xcorr plot look nice"""

    axis_kwargs = dict(c="k", lw=0.5, alpha=0.5)
    ax.axhline(0, **axis_kwargs)
    ax.axvline(0, **axis_kwargs)
    ax.set_ylim([-0.9, 1.1])
    ax.set_xlabel("Lag (years)")
    ax.set_xticks([-24, -12, 0, 12, 24], labels=[-2, -1, 0, 1, 2])
    ax.set_yticks([-0.5, 0, 0.5, 1])
    ax.set_ylabel("Correlation")
    return

Make the plot

In [ ]:
## compute cross-corr
xcorr = src.XRO.xcorr(idx, idx["T_3"], maxlags=24)

## plot result
fig, ax = plt.subplots(figsize=(3.5, 3))

## plot data
ax.plot(xcorr.lag, xcorr["T_3"], label=r"$T_3$", c="k")
ax.plot(xcorr.lag, xcorr["T_34"], label=r"$T_{3.4}$", c="k", ls="--")
ax.plot(xcorr.lag, xcorr["h"], label=r"$h$")
ax.plot(xcorr.lag, xcorr["h_w"], label=r"$h_w$")

## format plot
ax.set_title("Corr. with Niño 3")
ax.legend(prop=dict(size=8))
format_xcorr_ax(ax)

plt.show()

## Fit RO models

In [ ]:
## specify order of annual cycle, mask parameters
ac_order = 3
ac_mask_idx = [(1, 1)]  # epsilon
# ac_mask_idx = [(1,0), (1, 1)] # epsilon and F2
# ac_mask_idx = [(0,1),(1,0),(1, 1)] # all except R
# ac_mask_idx = None

## initialize model
model = src.XRO.XRO(ncycle=12, ac_order=ac_order, is_forward=True)

## get fit for reanalysis
fit_h = model.fit_matrix(idx[["T_34", "h"]], ac_mask_idx=ac_mask_idx)
fit_hw = model.fit_matrix(idx[["T_3", "h_w"]], ac_mask_idx=ac_mask_idx)

## extract params
p_h = model.get_RO_parameters(fit_h)
p_hw = model.get_RO_parameters(fit_hw)

In [ ]:
## colors for plotting
col = sns.color_palette()

## set up plot
fig, axs = plt.subplots(1, 3, figsize=(6.5, 2), layout="constrained")


## plot R and eps on LHS
axs[0].plot(p_h.cycle, p_h["R"], label=r"$h=\overline{h}$", c=col[0])
axs[0].plot(p_h.cycle, p_hw["R"], label=r"$h=h_w$", c=col[1])

axs[0].plot(p_h.cycle, -p_h["epsilon"], ls="--", lw=1, c=col[0])
axs[0].plot(p_h.cycle, -p_hw["epsilon"], ls="--", lw=1, c=col[1])

axs[1].plot(p_h.cycle, 2 * p_h["BJ_ac"], label=r"$h=\overline{h}$")
axs[1].plot(p_hw.cycle, 2 * p_hw["BJ_ac"], label=r"$h=h_w$")

axs[2].plot(p_h.cycle, fit_h["normxi_stdac"][0], label=r"$h=\overline{h}$")
axs[2].plot(p_h.cycle, fit_hw["normxi_stdac"][0], label=r"$h=\overline{h}$")
# axs[1].plot(p_hw.cycle, fit_hw["BJ_ac"], label=r"$h=h_w$")

for ax in axs:
    ## format
    ax.axhline(0, ls="-", c="k", lw=0.5)
    ax.set_xticks([1, 7, 12], labels=["Jan", "Jul", "Dec"])
    ax.set_ylim([-6, 5])

## label
axs[1].set_yticks([])
axs[2].set_yticks([])
axs[0].set_yticks([-3, 0, 3])
axs[0].legend(prop=dict(size=8))
axs[0].set_ylabel("year$^{-1}$")
axs[0].set_title(r"$R$ (solid) and $-\epsilon$ (dashed)")
axs[1].set_title(r"$2\left(R-\varepsilon\right)$")
axs[2].set_title(r"$\xi_T / \sigma_T$")

plt.show()

## check stats 

Generate simulations

In [ ]:
## specify random IC
x0 = idx.isel(time=rng.choice(np.arange(len(idx.time))))

## simulation specs
simulation_kwargs = dict(
    nyear=63,
    ncopy=1000,
    is_xi_stdac=False,
)

## do simulations
kwargs_h = dict(simulation_kwargs, fit_ds=fit_h, X0_ds=x0[["T_34", "h"]])
X_h = model.simulate(**kwargs_h)

kwargs_hw = dict(simulation_kwargs, fit_ds=fit_hw, X0_ds=x0[["T_3", "h_w"]])
X_hw = model.simulate(**kwargs_hw)

#### Seasonal synchronization

In [ ]:
### Set up plot
fig, axs = plt.subplots(1, 2, figsize=(4.5, 2), layout="constrained")

## plot RO with h (early period)
plot_data_early = src.utils.plot_seasonal_comp(
    axs[0],
    x0=idx.expand_dims("member"),
    x1=X_h,
    plot_kwargs0=dict(label="ORAS"),
    plot_kwargs1=dict(label="RO"),
    varname="T_34",
    use_quantile=True,
)

## plot RO with h (early period)
plot_data_early = src.utils.plot_seasonal_comp(
    axs[1],
    x0=idx.expand_dims("member"),
    x1=X_hw,
    plot_kwargs0=dict(label="ORAS"),
    plot_kwargs1=dict(label="RO"),
    varname="T_3",
    use_quantile=True,
)

## label
axs[1].legend(prop=dict(size=8))
axs[0].set_yticks([0, 1, 2])
axs[0].set_ylabel(r"$\sigma(T)$")
axs[0].set_title(r"Niño 3.4, $h=\overline{h}$")
axs[1].set_title(r"Niño 3, $h=h_w$")
for ax in axs:
    ax.set_ylim([0, 2])
    ax.set_xticks([1, 4, 12], labels=["Jan", "Apr", "Dec"])

plt.show()

#### Power spectrum

Compute

In [ ]:
## specify args for psd
psd_kwargs = dict(dim="time", dt=1 / 12, nw=5)

## compute PSD
compute_psd = lambda x: src.XRO_utils.pmtm(x, **psd_kwargs)
psd_oras = compute_psd(idx["T_34"])
psd_oras3 = compute_psd(idx["T_3"])
psd_RO_h = compute_psd(X_h["T_34"])
psd_RO_hw = compute_psd(X_hw["T_3"])

Plot

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6.5, 3), layout="constrained")

## plot data (T34)
src.utils.plot_psd(axs[0], psd_oras, label=r"ORAS", color="k")
src.utils.plot_psd(axs[0], psd_RO_h, label=r"RO", color=sns.color_palette()[1])

## plot data(T3)
src.utils.plot_psd(axs[1], psd_oras3, label=r"ORAS", color="k")
src.utils.plot_psd(axs[1], psd_RO_hw, label=r"RO", color=sns.color_palette()[1])

## label
axs[0].set_ylabel(r"PSD ($^{\circ}$C$^2$/cpm)")
axs[0].set_title(r"Niño 3.4 spectrum (using $h=\overline{h}$)")
axs[1].set_title(r"Niño 3 spectrum (using $h=h_w$)")
axs[0].legend(prop=dict(size=6))
axs[1].legend(prop=dict(size=6))


plt.show()

## Compute $R$ explicitly

In [ ]:
## compute gradients
grads = src.XRO.gradient(src.XRO._convert_to_numpy(idx))

## add to array
for i, n in enumerate(list(idx)):
    idx.update({f"{n}_grad": xr.DataArray(grads[i], coords=dict(time=idx.time))})


def get_stats(T_var, h_var, month):
    """Get statistics needed to compute R"""

    ## indexer
    t_idx = dict(time=idx.time.dt.month == month)

    ## get subset of array
    sel_vars = [T_var, h_var, f"{T_var}_grad", f"{h_var}_grad"]
    names = {
        T_var: "T",
        h_var: "h",
        f"{T_var}_grad": "T_grad",
        f"{h_var}_grad": "h_grad",
    }
    X = idx.isel(t_idx)[sel_vars].rename(names)

    ## compute stats
    sigma_T = np.std(X["T"].values)
    sigma_h = np.std(X["h"].values)
    r = scipy.stats.pearsonr(X["T"].values, X["h"].values)[0]

    ## compute covariance
    TtT = np.mean(X["T_grad"] * X["T"]).values.item() * np.pow(sigma_T, -2)
    Tth = (
        np.mean(X["T_grad"] * X["h"]).values.item()
        * np.pow(sigma_T, -1)
        * np.pow(sigma_h, -1)
    )

    return dict(r=r, TtT=TtT, Tth=Tth)


def get_R(T_var, h_var, month):
    """function to compute estimate of R for given variables and month"""

    ## compute stats
    stats = get_stats(T_var=T_var, h_var=h_var, month=month)
    r = stats["r"]
    TtT = stats["TtT"]
    Tth = stats["Tth"]

    ## compute estimate for R
    Rhat = 1 / (1 - r**2) * (TtT - r * Tth)

    return Rhat

\begin{align}
    R &= \frac{\sigma_T^{-1}}{1-r^2}\left[\left<T_t,\tilde{T}\right> - r\left<T_t,\tilde{h}\right>\right]
\end{align}

In [ ]:
#### compute
## kwargs
kwargs_h = dict(T_var="T_34", h_var="h")
kwargs_hw = dict(T_var="T_3", h_var="h_w")

## compute estimates
Rhats_h = np.array([get_R(month=m, **kwargs_h) for m in np.arange(1, 13)])
Rhats_hw = np.array([get_R(month=m, **kwargs_hw) for m in np.arange(1, 13)])


#### plot
colors = sns.color_palette()
fig, ax = plt.subplots(figsize=(4, 3))

## plot first version
ax.plot(p_h.cycle, Rhats_h, c=colors[0], ls="--")
ax.plot(p_h.cycle, p_h["R"], c=colors[0], label=r"$h$")

## plot second version
ax.plot(p_h.cycle, Rhats_hw, c=colors[1], ls="--")
ax.plot(p_h.cycle, p_hw["R"], c=colors[1], label=r"$h_w$")

## label
ax.legend(prop=dict(size=8))
ax.axhline(0, lw=1, c="k", zorder=0.5)

plt.show()

## Scatter plots

Look at relationship between $T_e$ and $\overline{h}-h_w$

In [ ]:
m = 12
t_idx = dict(time=idx.time.dt.month == m)

fig, ax = plt.subplots(figsize=(3, 3))
ax.scatter(
    idx["h"].isel(t_idx) - idx["h_w"].isel(t_idx),
    idx["T_3"].isel(t_idx),
    # -idx["h_w"].isel(t_idx), idx["T_3"].isel(t_idx)
)


## label
ax.set_xlabel(r"$\overline{h}-h_w$")
ax.set_ylabel(r"$T_e$")
kwargs = dict(c="k", lw=0.8, zorder=0.5)
ax.axvline(0, **kwargs)
ax.axhline(0, **kwargs)
plt.show()

Look at correlation as a function of month

In [ ]:
## get indices
Te = idx["T_3"]
dh = idx["h"] - idx["h_w"]

## compute correlation

In [ ]:
## empty array to hold result
corrs = []

for m in np.arange(1, 13):

    ## get boolean time indexer
    t_idx = dict(time=idx.time.dt.month == m)

    ## compute correlation
    corrs.append(xr.corr(Te.isel(t_idx), dh.isel(t_idx), dim="time").values)

## put in array
corrs = np.array(corrs)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
ax.bar(np.arange(1, 13), corrs)
ax.axhline(1, ls="--", c="k")
ax.set_xticks([1, 6, 12], labels=["Jan", "Jun", "Dec"])
ax.set_ylabel("Correlation")
plt.show()